# GCC118 - Programação Matemática  
## Universidade Federal de Lavras  

---

## Problema 3  

### Alunos:  
- **Gabriel Jardim de Souza** - 202310530  
- **Paulo Henrique Silveira dos Anjos** - 202310533


## Enunciado:

A cidade de União Paulista enfrenta uma séria carência orçamentária. Em busca de uma solução de longo prazo, a câmara de vereadores da cidade aprova uma melhoria da base da cobrança de impostos que prevê a condenação de uma área habitacional do centro da cidade e sua substituição por um conjunto habitacional moderno.  

O projeto envolve duas fases:  
1. demolição das casas que estão aquém do padrão para liberar terreno para o novo projeto;  
2. construção do novo conjunto urbano. A seguir daremos um resumo da situação.  

Um total de **300 casas** aquém do padrão podem ser demolidas. Cada casa ocupa um lote de **0,25 acres**. O custo de demolição de uma casa condenada é de **\$2.000**. Os tamanhos de lotes para domicílios (unidades) **simples, duplos, triplos e quádruplos** são de **0,18; 0,28; 0,4 e 0,5 acres**, respectivamente. Ruas, espaços abertos e instalações públicas ocupam **15% da área disponível**.  

---

**Tabela 1: Informações sobre as unidades habitacionais**

| Tipo de unidade | Tamanho do lote (acres) | Imposto cobrado por unidade (\$) | Custo de construção (\$) |
|-----------------|--------------------------|---------------------------------|--------------------------|
| **Simples**     | 0,18                     | 1.000                           | 50.000                   |
| **Dupla**       | 0,28                     | 1.900                           | 70.000                   |
| **Tripla**      | 0,40                     | 2.700                           | 130.000                  |
| **Quádrupla**   | 0,50                     | 3.400                           | 160.000                  |

---

No novo conjunto habitacional:  
- As **unidades triplas e quádruplas** representam no mínimo **25% do total**.  
- Unidades **simples** devem representar no mínimo **20%** de todas as unidades.  
- Unidades **duplas**, no mínimo **10%**.  

O **financiamento acordado com um banco local** será de no máximo **$15 milhões**.  

---

### Objetivo

Quantas unidades de cada tipo devem ser construídas para **maximizar a arrecadação de impostos**?  


## Instalação da biblioteca PuLP

In [1]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 62.1 MB/s eta 0:00:00


## Declaração dos parâmetros

* $max\_demolidas$: número máximo de casas que podem ser demolidas (300).  
* $acre\_por\_casa\_demolida$: área (em acres) ocupada por cada casa a ser demolida (0,25).  
* $fracao\_rua$: fração da área total destinada a ruas e espaços públicos (0,15).  
* $custo\_por\_casa\_demolida$: custo de demolição por casa (\$2.000).  

* $acre\_lotes$: lista com a área necessária para cada tipo de unidade habitacional:  
  - Simples: 0,18 acres  
  - Dupla: 0,28 acres  
  - Tripla: 0,40 acres  
  - Quádrupla: 0,50 acres  

* $imposto\_lotes$: lista com o valor do imposto arrecadado por unidade habitacional:  
  - Simples: \$1.000  
  - Dupla: \$1.900  
  - Tripla: \$2.700  
  - Quádrupla: \$3.400  

* $custo\_construcao$: lista com o custo de construção de cada tipo de unidade:  
  - Simples: \$50.000  
  - Dupla: \$70.000  
  - Tripla: \$130.000  
  - Quádrupla: \$160.000  

* $fin\_max$: valor máximo do financiamento disponível (\$15.000.000).  


In [2]:
max_demolidas = 300
acre_por_casa_demolida = 0.25
fracao_rua = 0.15
custo_por_casa_demolida = 2000

acre_lotes = [
    0.18,
    0.28,
    0.4,
    0.5
]

imposto_lotes = [
    1000,
    1900,
    2700,
    3400
]

custo_construcao = [
    50000,
    70000,
    130000,
    160000
]

fin_max = 15000000

## Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem('Problema3_Construcao_Uniao_Paulista', pulp.LpMaximize)

## Variáveis de decisão

* $num\_casas\_demolidas \ge 0$: número de casas a serem demolidas, inteiro, limitado por $max\_demolidas$.  
* $unidades[i] \ge 0$: número de unidades residenciais do tipo $i$ a serem construídas, $i \in \{0,1,2,3\}$, inteiro.  
  * $i=0$: unidades simples  
  * $i=1$: unidades duplas  
  * $i=2$: unidades triplas  
  * $i=3$: unidades quadruplas



In [4]:
num_casas_demolidas = pulp.LpVariable("num_casas_demolidas", lowBound=0, upBound=max_demolidas, cat='Integer')
unidades = [pulp.LpVariable(f"unidade_{i+1}", lowBound=0, cat='Integer') for i in range(4)]

## Função objetivo

* Maximização da arrecadação com impostos:  $\max \sum_{i=0}^{3} unidades[i] \cdot imposto\_lotes[i]$

In [5]:
modelo += pulp.lpSum([unidades[i] * imposto_lotes[i] for i in range(4)])

## Restrições

* Restrição de área total disponível: $\sum_{i=0}^{3} unidades[i] \cdot acre\_lotes[i] \le num\_casas\_demolidas \cdot acre\_por\_casa\_demolida \cdot (1 - fracao\_rua)$.

In [6]:
modelo += pulp.lpSum([unidades[i] * acre_lotes[i] for i in range(4)]) <= num_casas_demolidas * acre_por_casa_demolida * (1 - fracao_rua)

* Restrição de orçamento / financiamento máximo: $\sum_{i=0}^{3} unidades[i] \cdot custo\_construcao[i] + num\_casas\_demolidas \cdot custo\_por\_casa\_demolida \le fin\_max$.  

In [7]:
modelo += pulp.lpSum([unidades[i] * custo_construcao[i] for i in range(4)]) + num_casas_demolidas * custo_por_casa_demolida <= fin_max

* Restrições de proporção das unidades:  
  * $unidades[0] \ge 0.2 \cdot \sum_{i=0}^{3} unidades[i]$ (simples)  
  * $unidades[1] \ge 0.1 \cdot \sum_{i=0}^{3} unidades[i]$ (duplas)  
  * $unidades[2] + unidades[3] \ge 0.25 \cdot \sum_{i=0}^{3} unidades[i]$ (triplas + quadruplas)  

In [8]:
un_totais = pulp.lpSum(unidades)
modelo += unidades[0] >= 0.2 * un_totais
modelo += unidades[1] >= 0.1 * un_totais
modelo += (unidades[2] + unidades[3]) >= 0.25 * un_totais

### Resolvendo o problema

In [9]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [14]:
print('Status: ', pulp.LpStatus[status])
print('Função objetivo: $ {:,.2f}'.format(pulp.value(modelo.objective)))

descricoes = ["unidades simples", "unidades duplas", "unidades triplas", "unidades quadruplas"]

for desc, u in zip(descricoes, unidades):
    print(f"{desc} = {u.varValue}")

print(f"{num_casas_demolidas.name} = {num_casas_demolidas.varValue}")

gasto_financiamento = pulp.value(pulp.lpSum([unidades[i].varValue * custo_construcao[i] for i in range(4)])) \
                      + num_casas_demolidas.varValue * custo_por_casa_demolida

print(f"Gasto total de financiamento: $ {gasto_financiamento:,.2f}")


Status:  Optimal
Função objetivo: $ 343,700.00
unidades simples = 36.0
unidades duplas = 98.0
unidades triplas = 45.0
unidades quadruplas = 0.0
num_casas_demolidas = 245.0
Gasto total de financiamento: $ 15,000,000.00
